In [14]:
def defineTagWorkingArea(orderOfTags, dxfWorkingArea):
    anchorXY = dict()
#     dxfWorkingArea = sorted(dxfWorkingArea, key=itemgetter(0,1))
    
    for i in range(len(orderOfTags)):
        anchorXY.update({orderOfTags[i]: dxfWorkingArea[i]})
    
    return anchorXY

In [1]:
def calculateLength(e):
    xy = e.dxf.end - e.dxf.start
    length = math.sqrt(xy[0]**2+xy[1]**2)

    return length

def print_entity(e):
    print("start point: %s\n" % e.dxf.start)
    print("end point: %s\n" % e.dxf.end)
    print("length: %s\n" % calculateLength(e))

def getPoints(e):
    # originalLines.append((e.dxf.start, e.dxf.end))
    originalLines.append(e)
    originalPoints.append(e.dxf.start)
    originalPoints.append(e.dxf.end)
    originalPointsVector.append((e.dxf.start.x, e.dxf.start.y))
    originalPointsVector.append((e.dxf.end.x, e.dxf.end.y))


def getPointsInConvex(allPoints, convex):
    innerPoints = list()
    for e in allPoints:
        if edf.math.is_point_in_polygon_2d(e, convex):
            innerPoints.append(e)
    return innerPoints

def getWorkingAreaLines(workingArea, originalLines):
    workingLines = list()
    for point in workingArea:
        for line in originalLines:
            if edf.math.is_point_on_line_2d(point, (line.dxf.start.x, line.dxf.start.y), (line.dxf.end.x, line.dxf.end.y)):
                workingLines.append(line)
    return list(set(workingLines))
    

def constructPolygon(obstacleLines):
    obstaclePoints = list()
    for line in obstacleLines:
        obstaclePoints.append(line.dxf.start)
        obstaclePoints.append(line.dxf.end)
    
    polygon = edf.math.convex_hull_2d(obstaclePoints)
    
    print(polygon)
    return polygon
    
def getObstaclePolygons(innerLines, originalPoints):
    obstacles = list()
    for i in range(0, len(innerLines), 1):
        if i == len(innerLines)-1:
                break
        obstacle = list()
        firstLine = innerLines[i]
        if len(obstacles) > 0:
            start = firstLine.dxf.start
            end = firstLine.dxf.end
            print(obstacles[-1])
            convex = constructPolygon(obstacles[-1])
            if firstLine in obstacles[-1]:
                continue
            elif edf.math.is_point_in_polygon_2d(start, convex) in [0,1] and edf.math.is_point_in_polygon_2d(end, convex) in [0,1]:
                continue
            
            
        obstacle.append(firstLine)
        for j in range(1, len(innerLines), 1):
            secondLine = innerLines[j]
            intersection = edf.math.intersection_line_line_2d((firstLine.dxf.start, firstLine.dxf.end), \
                                                              (secondLine.dxf.start, secondLine.dxf.end))
        
            if intersection in originalPoints:
                obstacle.append(secondLine)
        
        if len(obstacle) > 0:
            obstacles.append(obstacle)
            
    return obstacles

In [9]:
import ezdxf as edf
import math
from operator import itemgetter

In [16]:
doc = edf.readfile('/Users/lawrence/Desktop/Drawing12 - office table test.dxf')
msp = doc.modelspace()

originalLines = list()
originalPoints = list()
originalVectorPoints = list()

for e in msp.query('LINE'):
    originalLines.append(e)
    originalVectorPoints.append(e.dxf.start)
    originalVectorPoints.append(e.dxf.end)
    originalPoints.append((e.dxf.start.x, e.dxf.start.y))
    originalPoints.append((e.dxf.end.x, e.dxf.end.y))
    
workingArea = edf.math.convex_hull_2d(originalVectorPoints)
# innerPoints = getPointsInConvex(originalVectorPoints, workingArea)

# workingLines = getWorkingAreaLines(workingArea, originalLines)
# innerLines = [line for line in originalLines if line not in workingLines]
# obstacles = getObstaclePolygons(innerLines, originalPoints)

In [18]:
defineTagWorkingArea(['C', 'B', 'A'], workingArea)

{'C': Vec2(19.19984935277204, 14.54758706799748),
 'B': Vec2(19.19984935277204, 15.44758706799748),
 'A': Vec2(21.35185314849843, 18.10033676686962)}

In [20]:
def parseDeca(decaData, anchorXY):
    baseX = tagXY['A'][0]
    baseY = tagXY['A'][1]
    
    tagX = decaData['TAG'][0]
    tagY = decaData['TAG'][1]
    
    tagDXFX = baseX + tagX
    tagDXFY = baseY + tagY
    
    tagPosition = edf.Math.Vec2(tagDXFX, tagDXFY)
    
    return tagPosition